# 📊 Information Conductivity Parameter Sweep Demo

**Da-P_Satulon Research Project**  
Interactive demonstration of cellular automata and information conductivity analysis.

This notebook provides:
- Quick parameter sweep demonstration
- Interactive visualization
- Method comparison
- Export capabilities for publication


In [ ]:
# Setup and imports
import sys
import os
sys.path.append('../code/ca_2d')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
import seaborn as sns

# Da-P_Satulon imports
from grid import CA2D
from info_cond import calculate_information_conductivity
from __init__ import create_ca

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Setup complete! Ready for CA experiments.")

## 🚀 Quick Demo: Single CA Run

Let's start with a single cellular automaton to understand the basics.

In [ ]:
# Create and run a single CA
print("Creating CA with grid size 30×30...")
ca = create_ca(grid_size=30, interaction_strength=0.5, seed=42)

print("Running 50 update steps...")
ca.update(50)

# Calculate different conductivity measures
methods = ['simple', 'entropy', 'gradient']
conductivities = {}

for method in methods:
    cond = calculate_information_conductivity(ca.grid, method=method)
    conductivities[method] = cond
    print(f"{method.capitalize()} conductivity: {cond:.4f}")

# Visualize the final grid state
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Initial state
axes[0].imshow(ca.history[0], cmap='viridis', vmin=0, vmax=1)
axes[0].set_title('Initial State')
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')

# Final state
axes[1].imshow(ca.grid, cmap='viridis', vmin=0, vmax=1)
axes[1].set_title('Final State (t=50)')
axes[1].set_xlabel('X')
axes[1].set_ylabel('Y')

# Conductivity comparison
methods_list = list(conductivities.keys())
values = list(conductivities.values())
bars = axes[2].bar(methods_list, values, color=['skyblue', 'orange', 'green'])
axes[2].set_title('Conductivity by Method')
axes[2].set_ylabel('Information Conductivity')
axes[2].grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, values):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                f'{value:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"\n📈 CA evolved from {len(ca.history)} states")
print(f"🎯 Interaction strength: {ca.interaction_strength}")
print(f"📏 Grid size: {ca.grid_size}")

## 📊 Parameter Sweep Analysis

Now let's perform a systematic parameter sweep to understand how interaction strength affects information conductivity.

In [ ]:
# Parameter sweep configuration
interaction_values = np.linspace(0.1, 1.0, 10)
grid_size = 40
iterations = 80
random_seed = 42

print(f"🔬 Running parameter sweep:")
print(f"   Interaction range: {interaction_values[0]:.1f} - {interaction_values[-1]:.1f}")
print(f"   Grid size: {grid_size}×{grid_size}")
print(f"   Iterations per experiment: {iterations}")
print(f"   Total experiments: {len(interaction_values)}")

# Store results
results = []

# Run experiments with progress bar
for i, interaction in enumerate(tqdm(interaction_values, desc="Running experiments")):
    # Create CA with different seed for each experiment
    ca = create_ca(grid_size=grid_size, 
                   interaction_strength=interaction, 
                   seed=random_seed + i)
    
    # Run simulation
    ca.update(iterations)
    
    # Calculate conductivity using different methods
    methods_data = {}
    for method in ['simple', 'entropy', 'gradient']:
        cond = calculate_information_conductivity(ca.grid, method=method)
        methods_data[f'conductivity_{method}'] = cond
    
    # Calculate time series for temporal analysis
    time_series = []
    for state in ca.history:
        cond = calculate_information_conductivity(state, method='simple')
        time_series.append(cond)
    
    # Store comprehensive results
    result = {
        'experiment_id': i,
        'interaction_strength': interaction,
        'final_conductivity': time_series[-1],
        'mean_conductivity': np.mean(time_series),
        'std_conductivity': np.std(time_series),
        'conductivity_trend': time_series[-1] - time_series[0],
        **methods_data,
        'time_series': time_series
    }
    
    results.append(result)

# Convert to DataFrame for easy analysis
df = pd.DataFrame(results)
print(f"\n✅ Completed {len(results)} experiments!")
print(f"📊 Results DataFrame shape: {df.shape}")
df.head()

## 📈 Comprehensive Visualization

Let's create publication-quality visualizations.

In [ ]:
# Create comprehensive analysis plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Information Conductivity Analysis - Parameter Sweep Results', fontsize=16)

# 1. Mean conductivity vs interaction strength
ax1 = axes[0, 0]
ax1.errorbar(df['interaction_strength'], df['mean_conductivity'], 
             yerr=df['std_conductivity'], marker='o', capsize=5, 
             color='blue', label='Mean ± Std')
ax1.plot(df['interaction_strength'], df['final_conductivity'], 
         's-', alpha=0.7, color='red', label='Final')
ax1.set_xlabel('Interaction Strength')
ax1.set_ylabel('Information Conductivity')
ax1.set_title('Conductivity vs Interaction Strength')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Method comparison
ax2 = axes[0, 1]
ax2.plot(df['interaction_strength'], df['conductivity_simple'], 'o-', label='Simple', alpha=0.8)
ax2.plot(df['interaction_strength'], df['conductivity_entropy'], 's-', label='Entropy', alpha=0.8)
ax2.plot(df['interaction_strength'], df['conductivity_gradient'], '^-', label='Gradient', alpha=0.8)
ax2.set_xlabel('Interaction Strength')
ax2.set_ylabel('Conductivity')
ax2.set_title('Method Comparison')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Evolution trends
ax3 = axes[0, 2]
ax3.plot(df['interaction_strength'], df['conductivity_trend'], 'go-', linewidth=2)
ax3.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax3.set_xlabel('Interaction Strength')
ax3.set_ylabel('Conductivity Trend (Final - Initial)')
ax3.set_title('Temporal Evolution Trends')
ax3.grid(True, alpha=0.3)

# 4. Time series for selected experiments
ax4 = axes[1, 0]
selected_indices = [0, len(df)//4, len(df)//2, 3*len(df)//4, len(df)-1]
colors = plt.cm.viridis(np.linspace(0, 1, len(selected_indices)))

for idx, color in zip(selected_indices, colors):
    time_series = results[idx]['time_series']
    interaction = results[idx]['interaction_strength']
    ax4.plot(range(len(time_series)), time_series, 
             color=color, alpha=0.8, linewidth=2,
             label=f'ρ = {interaction:.2f}')

ax4.set_xlabel('Time Step')
ax4.set_ylabel('Information Conductivity')
ax4.set_title('Time Evolution (Selected Cases)')
ax4.legend()
ax4.grid(True, alpha=0.3)

# 5. Statistical distribution
ax5 = axes[1, 1]
ax5.hist(df['mean_conductivity'], bins=15, alpha=0.7, color='skyblue', edgecolor='black')
ax5.axvline(df['mean_conductivity'].mean(), color='red', linestyle='--', linewidth=2,
            label=f'Mean: {df["mean_conductivity"].mean():.3f}')
ax5.set_xlabel('Mean Conductivity')
ax5.set_ylabel('Frequency')
ax5.set_title('Conductivity Distribution')
ax5.legend()
ax5.grid(True, alpha=0.3)

# 6. Method correlation
ax6 = axes[1, 2]
methods_df = df[['conductivity_simple', 'conductivity_entropy', 'conductivity_gradient']]
correlation_matrix = methods_df.corr()
im = ax6.imshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
ax6.set_xticks(range(len(methods_df.columns)))
ax6.set_yticks(range(len(methods_df.columns)))
ax6.set_xticklabels(['Simple', 'Entropy', 'Gradient'])
ax6.set_yticklabels(['Simple', 'Entropy', 'Gradient'])
ax6.set_title('Method Correlation Matrix')

for i in range(len(methods_df.columns)):
    for j in range(len(methods_df.columns)):
        ax6.text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                ha="center", va="center", color="black")

plt.colorbar(im, ax=ax6, label='Correlation')
plt.tight_layout()
plt.show()

## 🔍 Statistical Analysis and Key Findings

In [ ]:
# Generate summary of key findings
print("🔬 KEY RESEARCH FINDINGS")
print("=" * 60)

# 1. Optimal interaction strength
best_interaction = df.loc[df['mean_conductivity'].idxmax(), 'interaction_strength']
best_conductivity = df['mean_conductivity'].max()
print(f"\n1. 🎯 OPTIMAL INTERACTION STRENGTH: {best_interaction:.3f}")
print(f"   • Achieves maximum information conductivity: {best_conductivity:.4f}")
print(f"   • {((best_conductivity - df['mean_conductivity'].min()) / df['mean_conductivity'].min() * 100):.1f}% improvement over minimum")

# 2. Method comparison
method_correlations = df[['conductivity_simple', 'conductivity_entropy', 'conductivity_gradient']].corrwith(df['interaction_strength'])
most_sensitive_method = method_correlations.abs().idxmax().replace('conductivity_', '')
print(f"\n2. 📏 METHOD SENSITIVITY ANALYSIS:")
print(f"   • Most sensitive to interaction strength: {most_sensitive_method.upper()}")
for method in ['simple', 'entropy', 'gradient']:
    corr = method_correlations[f'conductivity_{method}']
    print(f"   • {method.capitalize()} method correlation: {corr:+.3f}")

# 3. Temporal dynamics
avg_trend = df['conductivity_trend'].mean()
trend_direction = "increasing" if avg_trend > 0 else "decreasing"
print(f"\n3. ⏱️  TEMPORAL EVOLUTION:")
print(f"   • Average conductivity trend: {avg_trend:+.4f} ({trend_direction})")
print(f"   • Most stable system: ρ = {df.loc[df['std_conductivity'].idxmin(), 'interaction_strength']:.3f}")
print(f"   • Most dynamic system: ρ = {df.loc[df['std_conductivity'].idxmax(), 'interaction_strength']:.3f}")

# 4. Phase transition indicators
conductivity_diff = np.diff(df['mean_conductivity'])
max_change_idx = np.argmax(np.abs(conductivity_diff))
transition_point = df.loc[max_change_idx, 'interaction_strength']
print(f"\n4. 🔄 PHASE TRANSITION INDICATORS:")
print(f"   • Strongest conductivity change at ρ = {transition_point:.3f}")
print(f"   • Change magnitude: {conductivity_diff[max_change_idx]:+.4f}")

# 5. Research recommendations
print(f"\n5. 💡 RESEARCH RECOMMENDATIONS:")
print(f"   • Focus detailed studies around ρ = {best_interaction:.3f} ± 0.1")
print(f"   • Investigate transition region near ρ = {transition_point:.3f}")
print(f"   • Use {most_sensitive_method} method for parameter sensitivity studies")
print(f"   • Consider longer time series for trend analysis")

print(f"\n🎉 Analysis complete! Ready for publication in Da-P_Satulon paper.")

## 💾 Export for Publication

Save results and publication-quality figures.

In [ ]:
# Create export directory
export_dir = '../results/notebook_demo'
os.makedirs(export_dir, exist_ok=True)
plots_dir = os.path.join(export_dir, 'plots')
os.makedirs(plots_dir, exist_ok=True)

print("💾 Exporting results for publication...")

# 1. Save comprehensive data
csv_file = os.path.join(export_dir, 'parameter_sweep_results.csv')
df.to_csv(csv_file, index=False)

# 2. Create publication figure
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

ax.errorbar(df['interaction_strength'], df['mean_conductivity'], 
            yerr=df['std_conductivity'], 
            marker='o', markersize=8, capsize=5, capthick=2,
            linewidth=2, color='darkblue', 
            label='Mean ± Standard Deviation')

# Highlight optimal point
best_idx = df['mean_conductivity'].idxmax()
ax.scatter(df.loc[best_idx, 'interaction_strength'], 
          df.loc[best_idx, 'mean_conductivity'],
          color='red', s=100, marker='*', 
          label='Maximum Conductivity', zorder=5)

ax.set_xlabel('Interaction Strength (ρ)', fontsize=14)
ax.set_ylabel('Information Conductivity', fontsize=14)
ax.set_title('Information Conductivity vs Interaction Strength\nCellular Automata Parameter Sweep', 
             fontsize=16, pad=20)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

# Save publication plots
for fmt in ['png', 'pdf', 'svg']:
    plot_file = os.path.join(plots_dir, f'conductivity_analysis.{fmt}')
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')

plt.show()

# 3. Save metadata
import json
metadata = {
    'experiment_type': 'parameter_sweep_demo',
    'grid_size': grid_size,
    'iterations': iterations,
    'interaction_range': [float(interaction_values[0]), float(interaction_values[-1])],
    'num_experiments': len(interaction_values),
    'optimal_interaction': float(best_interaction),
    'max_conductivity': float(best_conductivity),
    'methods_used': ['simple', 'entropy', 'gradient'],
    'notebook_version': '1.0',
    'timestamp': pd.Timestamp.now().isoformat()
}

with open(os.path.join(export_dir, 'experiment_metadata.json'), 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✅ Export complete!")
print(f"📁 Files saved to: {export_dir}")
print(f"📊 Data: parameter_sweep_results.csv")
print(f"🎨 Plots: plots/conductivity_analysis.*")
print(f"⚙️  Metadata: experiment_metadata.json")
print(f"\n🎯 Ready for integration into Da-P_Satulon research paper!")